<a href="https://colab.research.google.com/github/whossssssss/ML1/blob/main/ML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install wandb

In [38]:
# Log in to your W&B account
import wandb
wandb.login(key='288ce8e729a0a9bd1d74ece18ca65eb5bf20044a')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [39]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

wandb: Currently logged in as: k-shi (moyuml). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_062250-0w7o53en
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run experiment_0
wandb: ⭐️ View project at https://wandb.ai/moyuml/basic-intro
wandb: 🚀 View run at https://wandb.ai/moyuml/basic-intro/runs/0w7o53en
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  acc ▁▂▆▅▅▆█▆
wandb: loss ▆█▂▃▁▂▃▁
wandb: 
wandb: Run summary:
wandb:  acc 0.78402
wandb: loss 0.13551
wandb: 
wandb: 🚀 View run experiment_0 at: https://wandb.ai/moyuml/basic-intro/runs/0w7o53en
wandb: Synced 4 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230919_062250-0w7o53en/logs
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_062254-ilr2m3e6
wandb: 

In [40]:
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [41]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })

    # Copy your config
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})

        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run
    wandb.finish()

wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_062430-8c36q7qy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rare-silence-1
wandb: ⭐️ View project at https://wandb.ai/moyuml/pytorch-intro
wandb: 🚀 View run at https://wandb.ai/moyuml/pytorch-intro/runs/8c36q7qy


100%|██████████| 9912422/9912422 [00:00<00:00, 146467614.72it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 42178166.37it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

wandb: 
wandb: Run history:
wandb:      train/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: train/example_ct ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: train/train_loss █▄▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▁▂▂▂▂▂▂▂▁▃▁▂▂▂▁▁▁▂▂▁▂▂▁
wandb: val/val_accuracy ▁▃▅▅▆▆▆██▇
wandb:     val/val_loss █▅▄▃▃▂▂▁▁▁
wandb: 
wandb: Run summary:
wandb:    test_accuracy 0.8
wandb:      train/epoch 10.0
wandb: train/example_ct 120000
wandb: train/train_loss 0.09478
wandb: val/val_accuracy 0.9435
wandb:     val/val_loss 0.16558
wandb: 
wandb: 🚀 View run balmy-dust-4 at: https://wandb.ai/moyuml/pytorch-intro/runs/d6bwslmu
wandb: Synced 5 W&B file(s), 1 media file(s), 257 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230919_062726-d6bwslmu/logs
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_062822-uj6pd8on
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dutiful-glade-5
wandb: ⭐️ View project at https://wandb.a

Train Loss: 0.461, Valid Loss: 0.355178, Accuracy: 0.90
Train Loss: 0.263, Valid Loss: 0.287393, Accuracy: 0.91
Train Loss: 0.259, Valid Loss: 0.262261, Accuracy: 0.92
Train Loss: 0.315, Valid Loss: 0.244580, Accuracy: 0.93
Train Loss: 0.364, Valid Loss: 0.228895, Accuracy: 0.93
Train Loss: 0.259, Valid Loss: 0.225515, Accuracy: 0.93
Train Loss: 0.235, Valid Loss: 0.213315, Accuracy: 0.93
Train Loss: 0.243, Valid Loss: 0.203905, Accuracy: 0.94
Train Loss: 0.336, Valid Loss: 0.201341, Accuracy: 0.94


wandb: Waiting for W&B process to finish... (success).


Train Loss: 0.259, Valid Loss: 0.205470, Accuracy: 0.93


wandb: 
wandb: Run history:
wandb:      train/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: train/example_ct ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: train/train_loss █▅▄▃▃▃▂▃▃▃▂▃▃▂▂▂▂▁▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁
wandb: val/val_accuracy ▁▃▅▆▇▇███▇
wandb:     val/val_loss █▅▄▃▂▂▂▁▁▁
wandb: 
wandb: Run summary:
wandb:    test_accuracy 0.8
wandb:      train/epoch 10.0
wandb: train/example_ct 120000
wandb: train/train_loss 0.25925
wandb: val/val_accuracy 0.933
wandb:     val/val_loss 0.20547
wandb: 
wandb: 🚀 View run dutiful-glade-5 at: https://wandb.ai/moyuml/pytorch-intro/runs/uj6pd8on
wandb: Synced 5 W&B file(s), 1 media file(s), 257 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230919_062822-uj6pd8on/logs
